In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import extinction
from astroquery.irsa_dust import IrsaDust
import astropy.coordinates as coord
import astropy.units as u

def preprocess(filename, bin=False):
    #lightcurve data
    curve = pd.read_csv("./TESS_data/light_curves_fausnaugh/" + filename, delim_whitespace=True)
    curve_name = filename.split("_")[1]
    #information about transient
    curve_meta = transients[transients['IAU'] == curve_name]

    #convert time to relative to discovery
    curve['relative_time'] = curve['TJD'] - curve_meta["TJD_discovery"][0]
    curve.index = pd.TimedeltaIndex(curve['relative_time'], unit="D")

    #sigma clipping

    #sub bg flux

    #correct milky way extinction

    #bin
    if bin:
        pass

    #normalize

    return curve


transients = pd.read_csv("./TESS_data/AT_count_transients_s1-47 (4).txt", names=["sector", "ra", "dec", "mag", "TJD_discovery", "type" ,"class", "IAU", "survey", "cam", "ccd", "col", "row"], delim_whitespace=True)
light_curves = os.listdir("./TESS_data/light_curves_fausnaugh")

for curve_file in light_curves:
    processed = preprocess(curve_file)




C:\Users\Hali Huang\AppData\Local\Temp\ipykernel_16760\4201811503.py:6: DeprecationWarning: the ``irsa_dust`` module has been moved to astroquery.ipac.irsa.irsa_dust, please update your imports.
  from astroquery.irsa_dust import IrsaDust


lc_2018eel_cleaned
lc_2018emt_cleaned
lc_2018eny_cleaned
lc_2018eoa_cleaned
lc_2018eoc_cleaned
lc_2018eod_cleaned
lc_2018eph_cleaned
lc_2018eqo_cleaned
lc_2018esp_cleaned
lc_2018esw_cleaned
lc_2018evo_cleaned
lc_2018exc_cleaned
lc_2018fbm_cleaned
lc_2018fdw_cleaned
lc_2018fdx_cleaned
lc_2018fgb_cleaned
lc_2018fhs_cleaned
lc_2018fhw_cleaned
lc_2018fhw_detrended_cleaned
lc_2018flj_cleaned
lc_2018fll_cleaned
lc_2018flq_cleaned
lc_2018flr_cleaned
lc_2018flt_cleaned
lc_2018flz_cleaned
lc_2018fmg_cleaned
lc_2018fmm_cleaned
lc_2018fmn_cleaned
lc_2018fns_cleaned
lc_2018fqn_cleaned
lc_2018ftf_cleaned
lc_2018fub_cleaned
lc_2018fvi_cleaned
lc_2018fvi_detrended_cleaned
lc_2018fwd_cleaned
lc_2018fwi_cleaned
lc_2018fwk_cleaned
lc_2018fwm_cleaned
lc_2018fwr_cleaned
lc_2018fxj_cleaned
lc_2018fxl_cleaned
lc_2018fzi_cleaned
lc_2018gey_cleaned
lc_2018gfj_cleaned
lc_2018gfk_cleaned
lc_2018gku_cleaned
lc_2018glp_cleaned
lc_2018glq_cleaned
lc_2018glv_cleaned
lc_2018glw_cleaned
lc_2018goa_cleaned
lc_2018gqc_